In [1]:
import pickle
import pandas as pd


with open('models/label_encoder_RoomType.pkl', 'rb') as le_file:
     label_encoder_roomtype = pickle.load(le_file)     

with open('models/best_model_0.740.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

In [2]:
df_test = pd.DataFrame({
    'Room Type': ['Entire home/apt', 'Private room', 'Shared room'],
    'Accommodates': [6, 2, 4],
    'Bedrooms': [3, 1, 1],
    'Cleaning Fee':[40,20,10],
    'Bathrooms':[2,1,1],
    'Guests Included':[2,1,1],
    'Host Listings Count':[17,16,5]
})

In [3]:
df_test['Room Type'] = label_encoder_roomtype['model'].transform(df_test['Room Type'])


In [4]:
predictions = model.predict(df_test)
print(predictions)

[LightGBM] [Warning] feature_fraction is set=0.767866318661752, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.767866318661752
[41.49548023 37.24776078 32.84312371]


*Resumen: El modelo LightGBM fue el de mejor resultado r2 de 0.7845, aún asi el modelo no es capaz de generalizar y no se pudo replicar esos resultados,por lo que se debe continuar ajustando parámetros como min_data_in_leaf y bagging_fraction, evaluar la creación de nuevas features para mejorar aún más el rendimiento.*

La diferencia de features entre el modelo .7845 y el resto es la inclusión de la varia Beds al modelo.

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import preprocessing
df_train=pd.read_csv('./data/airbnb_train.csv', sep=";",decimal='.' )
df_test=pd.read_csv('./data/airbnb_test.csv', sep=";",decimal='.' )

df_train=df_train.dropna(subset=['Price'])
df_test=df_test.dropna(subset=['Price'])
df_train=preprocessing.filtrar_columnas(df_train,['Cleaning Fee','Accommodates','Room Type','Host Listings Count','Bathrooms','Bedrooms','Guests Included','Price'])
df_train , imputer , xnumeric = preprocessing.imputer_data_numeric_2(df_train)
df_train , encoding= preprocessing.encoding_categorical_column_2(df_train)

df_test = df_test[df_train.columns.intersection(df_test.columns)]
df_test[xnumeric] = imputer.transform(df_test[xnumeric])

for col, encoder_info in encoding.items():
    if encoder_info['type'] == 'LabelEncoder':
        df_test[col] = encoder_info['model'].transform(df_test[col].astype(str)) if col in encoder_info['model'].classes_ else -1
        
X_train = df_train.drop(columns=["Price"]) 
y_train = df_train["Price"] 
X_test = df_test.drop(columns=["Price"])
y_test = df_test["Price"]
        
y_pred = model.predict(X_test)


r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R² del modelo: {r2:.4f}")
print(f"Error cuadrático medio (MSE): {mse:.4f}")

[LightGBM] [Warning] feature_fraction is set=0.767866318661752, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.767866318661752
R² del modelo: 0.7403
Error cuadrático medio (MSE): 1413.8822
